1. Setup

In [ ]:
!pip install gym_super_mario_bros==7.3.0 nes_py

In [ ]:
# Spiel importieren
import gym_super_mario_bros
# Joypad wrapper importieren
from nes_py.wrappers import JoypadSpace
# Vereinfachte Kontrolle importieren
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [ ]:
from gym_super_mario_bros.actions import COMPLEX_MOVEMENT

In [ ]:
##mögliche Tastenkombis
RIGHT_ONLY

In [ ]:
COMPLEX_MOVEMENT

In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode="human")
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
done = True
for step in range(100000):
# Start the game to begin with
    if done:
        env.reset()
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())
    done = terminated or truncated
    env.render()
env.close()

In [ ]:
env.close()

In [ ]:
##256 mögliche Tastenkombis für AI (schwer zu lernen)
env.action_space

In [ ]:
state = env.reset()

In [ ]:
env.step(1)[4]

2. Preprocess Environment

In [ ]:
# Install pytorch
!pip install torch torchvision torchaudio

In [ ]:
# Install stable baselines for RL stuff
!pip install stable-baselines3[extra]

In [ ]:
# Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

In [ ]:
# 1. Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode="human")
# 2. Simplify the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
JoypadSpace.reset = lambda self, **kwargs: self.env.reset(**kwargs)
# 3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# 4. Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
state = env.reset()

In [ ]:
state, reward, done, info = env.step([5])

In [ ]:
state.shape

In [ ]:
plt.imshow(state[0])

In [ ]:
state, reward, done, info = env.step([5])

In [ ]:
plt.figure(figsize=(20,16))
for idx in range(state.shape[3]):
    plt.subplot(1,4,idx+1)
    plt.imshow(state[0][:,:,idx])
plt.show()

2. Preprocess Environment

In [ ]:
# Import os for file path management
import os 
# Import PPO for algos
from stable_baselines3 import PPO
from stable_baselines3 import A2C
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train1/'
LOG_DIR = './logs1/'

In [ ]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:
# This is the AI model started
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=512)   

######MlpPolicy für anderes Vorgehen!!!!######

In [ ]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=10000, callback=callback)

In [ ]:
model.save('mlpmodel')

Testing

In [ ]:
# Load model
model = PPO.load('./train1/best_model_10000')

In [ ]:
model = A2C.load('./train2/best_model_5000000')

In [ ]:
state = env.reset()


In [ ]:
env.close()

In [ ]:
env.observation_space = (3, 240, 256)

In [ ]:
state = env.reset()

while True: 
    
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

In [ ]:
env.close()